In [2]:
# Put in names here
result_image_path = "results/basetest2/test_latest/images"

In [7]:
import torch
import os
import re
from PIL import Image
import cv2
import numpy as np
from skimage.measure import structural_similarity as ssim

ImportError: cannot import name 'structural_similarity' from 'skimage.measure' (/Users/mattmistele/miniconda3/envs/sketch2face/lib/python3.7/site-packages/skimage/measure/__init__.py)

In [ ]:
def make_image_path_pairs(dir):
    image_pairs = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    fake_regex = r'(.*)_fake_B.png'
    real_regex = r'(.*)_real_B.png'
    fake_paths = {}
    real_paths = {}

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            path = os.path.join(root, fname)
            match = re.search(fake_regex, fname)
            if match:
                key = match[1]
                if key in real_paths:
                    real_path = real_paths[key]
                    image_pairs.append((path, real_path))
                    del real_paths[key]
                    continue
                real_paths[key] = path
                continue

            match = re.search(real_regex, fname)
            if match:
                key = match[1]
                if key in fake_paths:
                    fake_path = fake_paths[key]
                    image_pairs.append((fake_path, path))
                    del fake_paths[key]
                    continue
                real_paths[key] = path
    return image_pairs

In [6]:
# image_pairs_PIL = [(Image.open(fake).convert('RGB'), Image.open(real).convert('RGB')) for fake, real in image_path_pairs]
image_path_pairs = make_image_path_pairs(result_image_path)
image_pairs = [(cv2.imread(fake), cv2.imread(real)) for fake, real in image_path_pairs]

NameError: name 'make_image_path_pairs' is not defined

In [4]:
def l1_distance(imageA, imageB):
    sq_error = np.sum(abs(imageA.astype("float") - imageB.astype("float")))
    return sq_error / float(imageA.shape[0] * imageB.shape[1] * imageB.shape[2])

def l2_distance(imageA, imageB):
    sq_error = np.sum((imageA.astype("float") - imageB.astype("float"))**2)
    return sq_error / float(imageA.shape[0] * imageB.shape[1] * imageB.shape[2])

In [5]:
l1_distances = []
l2_distances = []
ssims = []
for fake, real in image_pairs:
    l1_distances.append(l1_distance(fake, real))
    l2_distances.append(l2_distance(fake, real))
    ssims.append(ssim(fake, real))

avg_l1_dist = sum(l1_distances)/len(l1_distances)
avg_l2_dist = sum(l2_distances)/len(l2_distances)
avg_ssim = sum(ssims)/len(ssims)
print("Average L1 dist between the {} image pairs: {}".format(len(l1_distances), avg_l1_dist))
print("Average L2 dist between the {} image pairs: {}".format(len(l2_distances), avg_l2_dist))
print("Average SSIM between the {} image pairs: {}".format(len(ssims), avg_ssim))

NameError: name 'image_pairs' is not defined